In [1]:
%%time
# Approach 0 - no imports
counter = {}

with open("citibike.csv") as f:
    f.readline() # skip headers
    for line in f:
        station = line.split(',')[4]
        counter.setdefault(station, 0)
        counter[station] += 1

print(sorted(counter.items(), key=lambda s: s[1], reverse=True)[:5])

[('"W 20 St & 11 Ave"', 5983), ('"E 17 St & Broadway"', 5621), ('"Broadway & W 58 St"', 5401), ('"Broadway & E 14 St"', 5177), ('"Broadway & W 24 St"', 4955)]
CPU times: user 328 ms, sys: 23.2 ms, total: 351 ms
Wall time: 488 ms


In [2]:
%%time
# Approach 1 - CSV DictReader and Counter
from csv import DictReader
from collections import Counter

counter = Counter()
with open("citibike.csv") as f:
    for row in DictReader(f):
        counter[row['start station name']] += 1

print(counter.most_common(5))

[('W 20 St & 11 Ave', 5983), ('E 17 St & Broadway', 5621), ('Broadway & W 58 St', 5401), ('Broadway & E 14 St', 5177), ('Broadway & W 24 St', 4955)]
CPU times: user 1.15 s, sys: 18.5 ms, total: 1.17 s
Wall time: 1.17 s


In [3]:
%%time
# Approach 2 - CSV Tuple Reader and Counter with generator
from csv import reader
from collections import Counter

def station_names(csv_path):
    with open(csv_path) as f:
        for row in reader(f):
            yield row[4]

print(Counter(station_names("citibike.csv")).most_common(5))

[('W 20 St & 11 Ave', 5983), ('E 17 St & Broadway', 5621), ('Broadway & W 58 St', 5401), ('Broadway & E 14 St', 5177), ('Broadway & W 24 St', 4955)]
CPU times: user 637 ms, sys: 18.8 ms, total: 656 ms
Wall time: 679 ms


In [4]:
%%time
# Approach 3 - Pandas
import pandas as pd

df = pd.read_csv("citibike.csv")
print(df['start station name'].value_counts()[:5])

start station name
W 20 St & 11 Ave      5983
E 17 St & Broadway    5621
Broadway & W 58 St    5401
Broadway & E 14 St    5177
Broadway & W 24 St    4955
Name: count, dtype: int64
CPU times: user 2.24 s, sys: 318 ms, total: 2.56 s
Wall time: 867 ms


In [5]:
%%time
# Approach 4 - Polars
import polars as pl

df = pl.read_csv("citibike.csv")
print(df['start station name'].value_counts(sort=True)[:5])

shape: (5, 2)
┌────────────────────┬───────┐
│ start station name ┆ count │
│ ---                ┆ ---   │
│ str                ┆ u32   │
╞════════════════════╪═══════╡
│ W 20 St & 11 Ave   ┆ 5983  │
│ E 17 St & Broadway ┆ 5621  │
│ Broadway & W 58 St ┆ 5401  │
│ Broadway & E 14 St ┆ 5177  │
│ Broadway & W 24 St ┆ 4955  │
└────────────────────┴───────┘
CPU times: user 279 ms, sys: 54.8 ms, total: 334 ms
Wall time: 151 ms


In [6]:
%%time
# Approach 5 - DuckDB
import duckdb

print(duckdb.sql("""
    SELECT "start station name", count(*)
    FROM read_csv('citibike.csv')
    GROUP BY "start station name"
    ORDER BY 2 DESC
    LIMIT 5
""").show())

┌────────────────────┬──────────────┐
│ start station name │ count_star() │
│      varchar       │    int64     │
├────────────────────┼──────────────┤
│ W 20 St & 11 Ave   │         5983 │
│ E 17 St & Broadway │         5621 │
│ Broadway & W 58 St │         5401 │
│ Broadway & E 14 St │         5177 │
│ Broadway & W 24 St │         4955 │
└────────────────────┴──────────────┘

None
CPU times: user 324 ms, sys: 32.4 ms, total: 357 ms
Wall time: 174 ms
